In [282]:
import pandas as pd
import requests
# from bs4 import BeautifulSoup

import time
from datetime import date, datetime

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [167]:
# Download Json materials:

# !pip install flatten_json # install once and then disable this line
from pandas.io.json import json_normalize
from flatten_json import flatten

In [145]:
res=requests.get(url)
res.status_code

200

### create date and epoch time

In [146]:
date_table=pd.read_csv('./data/weather/date.csv')
date_table.shape

(1095, 2)

In [147]:
date_table.head()

,date,time
0,2016-03-01,12:30
1,2016-03-02,12:30
2,2016-03-03,12:30
3,2016-03-04,12:30
4,2016-03-05,12:30


In [148]:
# date ='2019-02-03'
# time ='12:30' #00:05
def epoch_time(df):
    date=df['date']
    time =df['time']
    year = int(date.split('-')[0])
    month = int(date.split('-')[1].lstrip('0'))
    day = int((date.split('-')[2].lstrip('0')))
    hours = int(time.split(':')[0])
    minutes = int(time.split(':')[1])
    epoch = (datetime(year, month, day, hours, minutes) - datetime(1970,1,1)).total_seconds()
    #return( year, month, day, hours, minutes)
    return(int(epoch))

In [149]:
date_table['epoch_time']=date_table.apply(lambda x: epoch_time(x),axis=1)

In [303]:
date_table.shape

(1095, 3)

In [150]:
date_table.head()

,date,time,epoch_time
0,2016-03-01,12:30,1456835400
1,2016-03-02,12:30,1456921800
2,2016-03-03,12:30,1457008200
3,2016-03-04,12:30,1457094600
4,2016-03-05,12:30,1457181000


### create the airport coordinates table

In [206]:
airport_coordinates=pd.DataFrame({'airport':['ATL','JFK','LAX','ORD','DFW','BOS'],
              'x':[33.636667,40.639722,33.9425,41.978611,32.896944,42.363056],
             'y':[-84.428056,-73.778889,-118.408056,-87.904722,-97.038056,-71.006389]}
             )
airport_coordinates

,airport,x,y
0,ATL,33.637,-84.428
1,JFK,40.640,-73.779
2,LAX,33.943,-118.408
3,ORD,41.979,-87.905
4,DFW,32.897,-97.038
5,BOS,42.363,-71.006


### scrape data from web site

In [330]:
weather=pd.DataFrame()
count=0
for i in airport_coordinates.index[:1]:
    for j in date_table.index[1022:]:
        try:
            url='https://api.darksky.net/forecast/7e45c53d35733182afc69407813a66ed/'+str(airport_coordinates.iloc[i,1])+','+str(airport_coordinates.iloc[i,2])+','+str(date_table.iloc[j,2])+'?exclude=currently,flags'
    #         print(url)
            res=requests.get(url)
            temp=res.json() # convert the data to json mode
            x =pd.DataFrame( temp['hourly']['data'])
            x['latitude']=temp['latitude']
            x['longitude']=temp['longitude']
            x['daily icon']=temp['daily']['data'][0]['icon']
            x['daily summary']=temp['daily']['data'][0]['summary']
            x['airport']=airport_coordinates.iloc[i,0]
            x['epoch_time']=date_table.iloc[j,2]
            weather=weather.append(x)
            time.sleep(2.5)
            count+=1
            print(count)
        except:
            print(f'error for index of date_table: {j} or exceeded daily free limit query!' )
            pass

error for index of date_table: 1022 or exceeded daily free limit query!
error for index of date_table: 1023 or exceeded daily free limit query!
error for index of date_table: 1024 or exceeded daily free limit query!
error for index of date_table: 1025 or exceeded daily free limit query!
error for index of date_table: 1026 or exceeded daily free limit query!
error for index of date_table: 1027 or exceeded daily free limit query!
error for index of date_table: 1028 or exceeded daily free limit query!
error for index of date_table: 1029 or exceeded daily free limit query!
error for index of date_table: 1030 or exceeded daily free limit query!
error for index of date_table: 1031 or exceeded daily free limit query!
error for index of date_table: 1032 or exceeded daily free limit query!
error for index of date_table: 1033 or exceeded daily free limit query!
error for index of date_table: 1034 or exceeded daily free limit query!
error for index of date_table: 1035 or exceeded daily free limit

In [321]:
weather.shape

(2911, 23)

In [322]:
weather

,airport,apparentTemperature,cloudCover,daily icon,daily summary,dewPoint,epoch_time,humidity,icon,latitude,...,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,ATL,76.530,0.700,rain,Rain in the evening.,72.540,1534595400,0.920,partly-cloudy-night,33.637,...,rain,1019.390,Humid and Mostly Cloudy,75.000,1534564800,0.000,3.540,214.000,6.290,3.350
1,ATL,75.770,0.690,rain,Rain in the evening.,72.390,1534595400,0.940,partly-cloudy-night,33.637,...,rain,1019.030,Humid and Mostly Cloudy,74.230,1534568400,0.000,3.790,217.000,5.890,3.310
2,ATL,75.180,0.790,rain,Rain in the evening.,72.370,1534595400,0.960,partly-cloudy-night,33.637,...,NaN,1018.870,Humid and Mostly Cloudy,73.610,1534572000,0.000,3.610,219.000,6.230,4.220
3,ATL,74.910,0.870,rain,Rain in the evening.,72.130,1534595400,0.960,partly-cloudy-night,33.637,...,NaN,1018.860,Humid and Mostly Cloudy,73.360,1534575600,0.000,3.780,231.000,8.600,4.680
4,ATL,74.670,0.690,rain,Rain in the evening.,71.570,1534595400,0.950,partly-cloudy-night,33.637,...,NaN,1018.680,Mostly Cloudy,73.200,1534579200,0.000,3.530,235.000,7.040,4.070
5,ATL,74.710,0.800,rain,Rain in the evening.,71.580,1534595400,0.950,partly-cloudy-night,33.637,...,NaN,1018.450,Mostly Cloudy,73.240,1534582800,0.000,3.980,241.000,6.640,3.690
6,ATL,74.930,0.900,rain,Rain in the evening.,71.530,1534595400,0.940,partly-cloudy-night,33.637,...,NaN,1018.320,Mostly Cloudy,73.480,1534586400,0.000,3.610,229.000,7.950,4.240
7,ATL,75.080,0.860,rain,Rain in the evening.,71.800,1534595400,0.940,partly-cloudy-night,33.637,...,NaN,1018.530,Mostly Cloudy,73.590,1534590000,0.000,3.490,219.000,6.160,3.270
8,ATL,75.640,0.960,rain,Rain in the evening.,72.030,1534595400,0.930,cloudy,33.637,...,NaN,1018.760,Humid and Overcast,74.150,1534593600,0.000,3.220,227.000,7.700,3.820
9,ATL,76.470,1.000,rain,Rain in the evening.,72.230,1534595400,0.910,cloudy,33.637,...,NaN,1018.770,Humid and Overcast,74.980,1534597200,1.000,3.550,223.000,6.640,3.830


In [323]:
date_table.iloc[1021,:]

date          2018-12-17
time               12:30
epoch_time    1545049800
Name: 1021, dtype: object

In [324]:
# weather.to_csv('./data/weather/ATL_1022_.csv',index=False)

In [305]:
# x=pd.read_csv('./data/weather/ATL_0_900.csv')
# x.shape

(21590, 24)

In [306]:
x

,airport,apparentTemperature,cloudCover,daily icon,daily summary,dewPoint,epoch_time,humidity,icon,latitude,...,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,ATL,50.400,0.260,rain,Heavy rain starting in the evening.,44.020,1456835400,0.790,partly-cloudy-night,33.637,...,NaN,1018.580,Partly Cloudy,50.400,1456808400,0.000,4.570,253.000,1.030,0.060
1,ATL,48.700,0.350,rain,Heavy rain starting in the evening.,43.890,1456835400,0.830,partly-cloudy-night,33.637,...,NaN,1018.720,Partly Cloudy,48.700,1456812000,0.000,3.130,270.000,0.990,0.020
2,ATL,47.860,0.010,rain,Heavy rain starting in the evening.,43.550,1456835400,0.850,clear-night,33.637,...,NaN,1018.450,Clear,47.860,1456815600,0.000,3.880,0.000,0.570,0.010
3,ATL,46.340,0.000,rain,Heavy rain starting in the evening.,43.040,1456835400,0.880,clear-night,33.637,...,NaN,1017.780,Clear,46.340,1456819200,0.000,3.600,312.000,0.510,0.020
4,ATL,45.700,0.240,rain,Heavy rain starting in the evening.,42.960,1456835400,0.900,clear-night,33.637,...,NaN,1017.490,Clear,45.700,1456822800,0.000,2.830,nan,0.420,0.000
5,ATL,45.500,0.230,rain,Heavy rain starting in the evening.,43.080,1456835400,0.910,clear-night,33.637,...,NaN,1017.500,Clear,45.500,1456826400,0.000,3.880,90.000,0.350,0.020
6,ATL,45.890,0.590,rain,Heavy rain starting in the evening.,43.430,1456835400,0.910,partly-cloudy-night,33.637,...,NaN,1017.850,Partly Cloudy,45.890,1456830000,0.000,4.580,220.000,0.740,0.050
7,ATL,47.900,0.900,rain,Heavy rain starting in the evening.,44.870,1456835400,0.890,partly-cloudy-night,33.637,...,NaN,1018.250,Mostly Cloudy,47.900,1456833600,0.000,3.650,244.000,1.030,0.100
8,ATL,49.880,1.000,rain,Heavy rain starting in the evening.,46.760,1456835400,0.890,cloudy,33.637,...,NaN,1018.590,Overcast,49.880,1456837200,0.000,4.260,232.000,1.740,0.530
9,ATL,54.530,0.970,rain,Heavy rain starting in the evening.,50.440,1456835400,0.860,cloudy,33.637,...,NaN,1018.750,Overcast,54.530,1456840800,1.000,4.030,212.000,3.840,1.610


# CREATE FUNCTONS AND LOOPS:
`this part is the pre work for the all above functions and loops. So this part is for user only not  for reader `

### Create body of the Loop:

In [165]:
temp=res.json()
temp

{'latitude': 33.636667,
 'longitude': -84.428056,
 'timezone': 'America/New_York',
 'hourly': {'summary': 'Rain starting in the afternoon, continuing until evening.',
  'icon': 'rain',
  'data': [{'time': 1456981200,
    'summary': 'Partly Cloudy',
    'icon': 'partly-cloudy-night',
    'precipIntensity': 0,
    'precipProbability': 0,
    'temperature': 39.02,
    'apparentTemperature': 39.02,
    'dewPoint': 29.44,
    'humidity': 0.68,
    'pressure': 1020.97,
    'windSpeed': 0.31,
    'windGust': 1.1,
    'windBearing': 331,
    'cloudCover': 0.27,
    'uvIndex': 0,
    'visibility': 4.33},
   {'time': 1456984800,
    'summary': 'Clear',
    'icon': 'clear-night',
    'precipIntensity': 0,
    'precipProbability': 0,
    'temperature': 38.26,
    'apparentTemperature': 38.26,
    'dewPoint': 29.9,
    'humidity': 0.72,
    'pressure': 1020.72,
    'windSpeed': 0.06,
    'windGust': 1.24,
    'windBearing': 3,
    'cloudCover': 0.07,
    'uvIndex': 0,
    'visibility': 3.58},
   {'

In [183]:
temp.keys()

dict_keys(['latitude', 'longitude', 'timezone', 'hourly', 'daily', 'offset'])

In [234]:
x =pd.DataFrame( temp['hourly']['data'])
x['latitude']=temp['latitude']
x['longitude']=temp['longitude']
x['daily icon']=temp['daily']['data'][0]['icon']
x['daily summary']=temp['daily']['data'][0]['summary']
x['airport']=airport_coordinates.iloc[0,0]
x['epoch_time']=date_table.iloc[0,2]
#[flatten(dic) for dic in x]
x

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,pressure,summary,...,visibility,windBearing,windGust,windSpeed,latitude,longitude,daily icon,daily summary,airport,epoch_time
0,39.020,0.270,29.440,0.680,partly-cloudy-night,0.000,0.000,NaN,1020.970,Partly Cloudy,...,4.330,331,1.100,0.310,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400
1,38.260,0.070,29.900,0.720,clear-night,0.000,0.000,NaN,1020.720,Clear,...,3.580,3,1.240,0.060,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400
2,37.620,0.030,30.880,0.760,clear-night,0.000,0.000,NaN,1020.800,Clear,...,4.260,89,0.950,0.130,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400
3,36.450,0.000,30.410,0.780,clear-night,0.000,0.000,NaN,1020.590,Clear,...,3.870,41,1.280,0.290,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400
4,36.530,0.000,30.160,0.770,clear-night,0.000,0.000,NaN,1020.210,Clear,...,3.130,53,1.120,0.210,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400
5,37.000,0.250,29.290,0.730,partly-cloudy-night,0.000,0.000,NaN,1019.780,Partly Cloudy,...,4.350,61,2.830,0.950,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400
6,36.860,0.140,28.440,0.710,clear-night,0.000,0.000,NaN,1019.840,Clear,...,4.530,78,4.320,1.060,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400
7,36.630,0.140,27.920,0.700,clear-night,0.000,0.000,NaN,1020.070,Clear,...,4.460,72,3.440,1.820,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400
8,35.780,0.270,28.380,0.670,partly-cloudy-day,0.000,0.000,NaN,1020.330,Partly Cloudy,...,4.380,93,8.440,3.590,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400
9,39.460,0.270,29.170,0.600,partly-cloudy-day,0.000,0.000,NaN,1020.160,Partly Cloudy,...,4.280,109,10.980,4.140,33.637,-84.428,rain,"Rain starting in the afternoon, continuing unt...",ATL,1456835400


,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,pressure,summary,...,visibility,windBearing,windGust,windSpeed,latitude,longitude,daily icon,daily summary,airport,epoch_time
0,51.230,1.000,48.520,0.900,cloudy,0.004,0.670,rain,1013.670,Overcast,...,4.380,281,7.170,2.680,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800


In [274]:
weather=x.iloc[[0]]
weather.drop(weather.index[:],inplace=True)
weather

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,pressure,summary,...,visibility,windBearing,windGust,windSpeed,latitude,longitude,daily icon,daily summary,airport,epoch_time


In [275]:
weather

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,pressure,summary,...,visibility,windBearing,windGust,windSpeed,latitude,longitude,daily icon,daily summary,airport,epoch_time


In [273]:
# weather=weather.append(x)
# weather

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,pressure,summary,...,visibility,windBearing,windGust,windSpeed,latitude,longitude,daily icon,daily summary,airport,epoch_time
0,51.230,1.000,48.520,0.900,cloudy,0.004,0.670,rain,1013.670,Overcast,...,4.380,281,7.170,2.680,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800
1,50.210,1.000,47.030,0.890,cloudy,0.001,0.170,rain,1014.510,Overcast,...,3.090,286,6.680,2.280,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800
2,48.580,0.520,45.310,0.870,partly-cloudy-night,0.001,0.120,rain,1014.740,Partly Cloudy,...,4.430,287,7.520,3.020,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800
3,46.380,0.210,43.790,0.870,clear-night,0.000,0.000,NaN,1014.880,Clear,...,4.210,302,9.250,3.480,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800
4,45.260,0.350,41.850,0.850,partly-cloudy-night,0.000,0.000,NaN,1014.870,Partly Cloudy,...,3.850,313,10.600,3.040,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800
5,41.240,0.420,38.560,0.820,partly-cloudy-night,0.000,0.000,NaN,1015.810,Partly Cloudy,...,3.790,311,11.210,4.560,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800
6,38.570,0.210,36.160,0.810,clear-night,0.000,0.000,NaN,1016.620,Clear,...,4.380,317,10.780,4.550,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800
7,36.510,0.730,33.990,0.800,partly-cloudy-night,0.000,0.000,NaN,1017.370,Mostly Cloudy,...,4.200,314,10.830,4.400,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800
8,35.480,0.770,33.140,0.790,partly-cloudy-day,0.000,0.000,NaN,1018.650,Mostly Cloudy,...,4.560,317,11.410,4.680,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800
9,35.160,0.850,31.850,0.740,partly-cloudy-day,0.000,0.000,NaN,1020.230,Mostly Cloudy,...,4.250,323,15.590,5.610,33.637,-84.428,partly-cloudy-day,Partly cloudy in the morning.,ATL,1456921800


In [170]:
json_normalize(temp)

,daily.data,hourly.data,hourly.icon,hourly.summary,latitude,longitude,offset,timezone
0,"[{'time': 1456981200, 'summary': 'Rain startin...","[{'time': 1456981200, 'summary': 'Partly Cloud...",rain,"Rain starting in the afternoon, continuing unt...",33.637,-84.428,-5,America/New_York


In [166]:
temp['latitude']

33.636667

In [101]:
c=1546360200 # '2019-01-01' '16:30'

l=[1546329600,1546333200,1546336800,1546340400,1546344000,1546347600,
   1546351200,1546354800,1546358400,1546362000,1546365600,1546369200,
   1546372800,1546376400,1546380000,1546383600,1546387200,1546390800,
   1546394400,1546398000,1546401600,1546405200,1546408800,1546412400
  ]

for i in l:
    s=(i-c)/3600
    print(s)

-8.5
-7.5
-6.5
-5.5
-4.5
-3.5
-2.5
-1.5
-0.5
0.5
1.5
2.5
3.5
4.5
5.5
6.5
7.5
8.5
9.5
10.5
11.5
12.5
13.5
14.5


In [86]:
c=1549197000 #'2019-02-03' '12:30'
l=[1549180800,1549184400,1549188000,
   1549256400,1549260000,1549263600
  
  ]

for i in l:
    s=(i-c)/3600
    print(s)

-4.5
-3.5
-2.5
16.5
17.5
18.5
